# BDCC HW: Task 3

Task: Implement a MapReduce job that identifies the 100 most followed users in the dataset.

Hint: We are not interested in creating lists of followers here. We just need to count the followers of each user in the Reduce phase. This is called the in-degree of a user. Moreover, a temporary data structure D of fixed 100 positions is required. This structure will be initially filled with the first 100 users that are processed in the Reduce phase. Then, the next users (101, 102, 103…) will replace a user in D, only if their in-degree is greater than the in-degree of the least-followed user in D. Notice that the ideal data structure for D is a min-heap (https://docs.python.org/3/library/heapq.html). However, it is totally acceptable to appropriately use a data structure such as a dictionary, or a list.

In [9]:
%%file src/task3.py
#!/usr/bin/env python3


from mrjob.job import MRJob
import heapq

from mrjob.step import MRStep

TOP_FOLLOWERS = 100

# Implement a MapReduce job that creates a list of followees for each user in the dataset.
class MostFollowed(MRJob):

    # Arg 1: self: the class itself (this)
    # Arg 2: Input key to the map function (here:none)
    # Arg 3: Input value to the map function (here:one line from the input file)
    def mapper(self, _, line):
        # yield (followee, 1) pair
        (follower, followee) = line.split()
        yield(followee, 1)


    def combiner(self, followee, follower_count):

        # yield sum of followers
        yield(followee, sum(follower_count))

    def reducer_init(self):
        self.heap = []

    # Arg 1: self: the class itself (this)
    # Arg 2: Input key to the reduce function (here: the key that was emitted by the mapper)
    # Arg 3: Input value to the reduce function (here: a generator object; something like a
    # sorted list of ALL values associated with the same key)
    def reducer(self, followee, follower_count):
        heapq.heappush(self.heap, (sum(follower_count), followee))
        
        if len(self.heap) > TOP_FOLLOWERS:
            heapq.heappop(self.heap)

    def reducer_final(self):
        for (follower_count, followee) in self.heap:
            yield (followee, follower_count)


    # Step 2 — The global TOP_FOLLOWERS needs to run.
    # The mapper outputs "TOP_FOLLOWERS" as the key and (count,word) as the value.
    # We put the count first so that it can be used directly as input to heapq.nlargest()
    def globalTop_mapper(self, followee, follower_count):
        yield "Top " + str(TOP_FOLLOWERS), (follower_count, followee)


    # The reducer ignores the key ("TOP_FOLLOWERS"), 
    # and just finds the largest of the values.
    def globalTop_reducer(self ,_, follower_count):
        for follower_count in heapq.nlargest(TOP_FOLLOWERS, follower_count):
            yield follower_count[1], follower_count[0]

    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer_init=self.reducer_init,
                   reducer=self.reducer,
                   reducer_final=self.reducer_final
                   ),

            MRStep(mapper=self.globalTop_mapper,
                   reducer=self.globalTop_reducer) 
        ]


if __name__ == '__main__':
    MostFollowed.run()

Overwriting src/task3.py


### Run in Standalone Mode

In [11]:
!python3 src/task3.py data/graph.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /var/folders/f7/py_w9f_n5fz4vg7_hty30b0m0000gn/T/task3.sheenafernandez.20210519.203332.109920
Running step 1 of 1...

Error while reading from /var/folders/f7/py_w9f_n5fz4vg7_hty30b0m0000gn/T/task3.sheenafernandez.20210519.203332.109920/step/000/reducer/00000/input:

Traceback (most recent call last):
  File "src/task3.py", line 53, in <module>
    MostFollowed.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/runn

### Run in the Hadoop cluster in a fully/pseudo distributed mode

In [18]:
!python3 src/task3.py -r hadoop data/graph.txt -o task3_output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in $PATH...
Falling back to 'hadoop'
Traceback (most recent call last):
  File "src/task1.py", line 31, in <module>
    Followers.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/job.py", line 616, in run
    cls().execute()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/job.py", line 687, in execute
    self.run_job()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/job.py", line 636, in run_job
    runner.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/runner.py", line 503, in run
    self._run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mrjob/hadoop.py", line 325, in _run
    self._find_binaries_and_jars()
  File "/Library/Frameworks/Python

### Copy the output from HDFS to local file system.

In [ ]:
!hdfs dfs -copyToLocal task2_output /home/bdccuser/bdcc-assignment1/output/task2